In [8]:
import hashlib
import pandas as pd

# Load the data (replace with the actual file path)
data = pd.read_csv(r'C:\Users\seang\OneDrive\cse 6242\project\CSE6242_Team148\notebooks\Spotifty_songs_CSE6242.csv')

# Function to generate a unique hash ID for each artist name
def generate_artist_id(name):
    return hashlib.md5(str(name).encode()).hexdigest()

# Step 1: Create separate dataframes for each table and save them as CSV files

# Albums table
albums_df = data[['album_name', 'release_date', 'album_image_url']].drop_duplicates().copy()
albums_df.insert(0, 'album_id', range(1, len(albums_df) + 1))
albums_df.to_csv('albums.csv', index=False)

# Tracks table
tracks_df = data[['track_id', 'track_name', 'popularity', 'duration_ms', 'explicit',
                  'track_external_url', 'type', 'id', 'uri', 'track_href', 'analysis_url',
                  'time_signature', 'album_name']].drop_duplicates(subset="track_id").copy()
tracks_df = tracks_df.merge(albums_df[['album_id', 'album_name']], 
                                          left_on='album_name', right_on='album_name')
tracks_df.to_csv('tracks.csv', index=False)

# Artists table with unique hashed IDs
artists_df = data[['artists', 'artist_popularity', 'artist_followers', 
                   'artist_image_url', 'artist_external_url']].copy()
# Split the artists
artists_df['artists'] = artists_df['artists'].str.split(',')
artists_df = artists_df.explode('artists')
artists_df["artists"] = artists_df['artists'].str.strip() # Remove extra spaces

# Drop duplicates by artist name to ensure each artist has only one row
artists_df = artists_df.drop_duplicates(subset="artists")
# Generate a unique hash ID for each artist name
artists_df['artist_id'] = artists_df['artists'].apply(generate_artist_id)
artists_df.rename(columns={'artists': 'artist_name'}, inplace=True)
artists_df.to_csv('artists.csv', index=False)

# Track_Artists table (many-to-many relationship)
track_artists_df = data[['track_id', 'artists']].copy()
# Split and strip artist names
track_artists_df['artists'] = track_artists_df['artists'].str.split(',')
track_artists_df = track_artists_df.explode('artists')
track_artists_df['artists'] = track_artists_df['artists'].str.strip()

# Merge with artists_df to replace artist names with artist IDs
track_artists_df = track_artists_df.merge(artists_df[['artist_id', 'artist_name']], 
                                          left_on='artists', right_on='artist_name')
track_artists_df = track_artists_df[['track_id', 'artist_id']]
track_artists_df.drop_duplicates(inplace=True)
track_artists_df.to_csv('track_artists.csv', index=False)

# Track Features table
track_features_df = data[['track_id', 'danceability', 'energy', 'key', 'loudness', 'mode',
                          'speechiness', 'acousticness', 'instrumentalness', 'liveness',
                          'valence', 'tempo']].drop_duplicates(subset="track_id").copy()
track_features_df.to_csv('track_features.csv', index=False)


In [1]:
import os
import psycopg2
from dotenv import load_dotenv
import pandas as pd

# Load environment variables
load_dotenv(r"C:\Users\seang\OneDrive\spotify-graph-playlist\config\dev.env")

True

In [6]:
import sqlite3
import pandas as pd

# Define the paths to your CSV files
tracks_csv = r'C:\Users\seang\OneDrive\cse 6242\project\CSE6242_Team148\assets\tracks.csv'
albums_csv = r'C:\Users\seang\OneDrive\cse 6242\project\CSE6242_Team148\assets\albums.csv'
artists_csv = r'C:\Users\seang\OneDrive\cse 6242\project\CSE6242_Team148\assets\artists.csv'
track_artists_csv = r'C:\Users\seang\OneDrive\cse 6242\project\CSE6242_Team148\assets\track_artists.csv'
track_features_csv = r'C:\Users\seang\OneDrive\cse 6242\project\CSE6242_Team148\assets\track_features.csv'


# Connect to SQLite database (or create it)
conn = sqlite3.connect(r'C:\Users\seang\OneDrive\spotify-graph-playlist\src\assets\music_data.db')
cursor = conn.cursor()

# Create tables and load data

# Tracks Table
cursor.execute("DROP TABLE IF EXISTS tracks")
cursor.execute('''CREATE TABLE IF NOT EXISTS tracks (
    track_id TEXT PRIMARY KEY,
    track_name TEXT,
    popularity INTEGER,
    duration_ms INTEGER,
    explicit BOOLEAN,
    track_external_url TEXT,
    type TEXT,
    id TEXT,
    uri TEXT,
    track_href TEXT,
    analysis_url TEXT,
    time_signature INTEGER,
    album_id TEXT
)''')
df = pd.read_csv(tracks_csv)
df.to_sql('tracks', conn, if_exists='replace', index=False)
df.to_pickle(r'C:\Users\seang\OneDrive\spotify-graph-playlist\src\assets\tracks.pkl')
# Albums Table
cursor.execute("DROP TABLE IF EXISTS albums")
cursor.execute('''CREATE TABLE IF NOT EXISTS albums (
    album_id TEXT PRIMARY KEY,
    album_name TEXT,
    release_date TEXT,
    album_image_url TEXT
)''')
df = pd.read_csv(albums_csv)
df.to_sql('albums', conn, if_exists='replace', index=False)
df.to_pickle(r'C:\Users\seang\OneDrive\spotify-graph-playlist\src\assets\albums.pkl')

# Artists Table with Auto-Incrementing artist_id
cursor.execute("DROP TABLE IF EXISTS artists")
cursor.execute('''CREATE TABLE IF NOT EXISTS artists (
    artist_id TEXT PRIMARY KEY,
    artist_name TEXT,
    artist_popularity INTEGER,
    artist_followers INTEGER,
    artist_image_url TEXT,
    artist_external_url TEXT
)''')
df = pd.read_csv(artists_csv)
df.to_sql('artists', conn, if_exists='replace', index=False)
df.to_pickle(r'C:\Users\seang\OneDrive\spotify-graph-playlist\src\assets\artists.pkl')

# Track_Artists Table (Many-to-Many relationship) with foreign keys
cursor.execute("DROP TABLE IF EXISTS track_artists")
cursor.execute('''CREATE TABLE IF NOT EXISTS track_artists (
    track_id TEXT,
    artist_id INTEGER,
    FOREIGN KEY (track_id) REFERENCES tracks (track_id),
    FOREIGN KEY (artist_id) REFERENCES artists (artist_id)
)''')
df = pd.read_csv(track_artists_csv)
df.to_sql('track_artists', conn, if_exists='replace', index=False)
df.to_pickle(r'C:\Users\seang\OneDrive\spotify-graph-playlist\src\assets\track_artists.pkl')

# Track Features Table
cursor.execute("DROP TABLE IF EXISTS track_features")
cursor.execute('''CREATE TABLE IF NOT EXISTS track_features (
    track_id TEXT PRIMARY KEY,
    danceability REAL,
    energy REAL,
    key INTEGER,
    loudness REAL,
    mode INTEGER,
    speechiness REAL,
    acousticness REAL,
    instrumentalness REAL,
    liveness REAL,
    valence REAL,
    tempo REAL,
    FOREIGN KEY (track_id) REFERENCES tracks (track_id)
)''')
df = pd.read_csv(track_features_csv)
df.to_sql('track_features', conn, if_exists='replace', index=False)
df.to_pickle(r'C:\Users\seang\OneDrive\spotify-graph-playlist\src\assets\track_features.pkl')

# Create Artist_profiles Table
cursor.execute("DROP TABLE IF EXISTS artist_profiles")
cursor.execute('''CREATE TABLE IF NOT EXISTS artist_profiles (
    artist_id INTEGER PRIMARY KEY,
    danceability REAL,
    energy REAL,
    key INTEGER,
    loudness REAL,
    mode INTEGER,
    speechiness REAL,
    acousticness REAL,
    instrumentalness REAL,
    liveness REAL,
    valence REAL,
    tempo REAL,
    FOREIGN KEY (artist_id) REFERENCES artists (artist_id)
)''')

# Load artist and track feature data
track_features_df = pd.read_csv(track_features_csv)
track_artists_df = pd.read_csv(track_artists_csv)

# Merge track features with artist-track mapping
artist_features_df = track_artists_df.merge(track_features_df, on='track_id')

# Aggregate features by artist
artist_profiles_df = artist_features_df.groupby('artist_id').agg({
    'danceability': 'mean',
    'energy': 'mean',
    'loudness': 'mean',
    'speechiness': 'mean',
    'acousticness': 'mean',
    'instrumentalness': 'mean',
    'liveness': 'mean',
    'valence': 'mean',
    'tempo': 'mean',
}).reset_index()

# Insert aggregated data into the artist_profiles table
artist_profiles_df.to_sql('artist_profiles', conn, if_exists='replace', index=False)

# Commit changes and close the connection
conn.commit()
conn.close()

In [40]:
import os
import pandas as pd
import plotly.express as px
import streamlit as st
from pathlib import Path


conn = sqlite3.connect(r'C:\Users\seang\OneDrive\cse 6242\project\CSE6242_Team148\assets\music_data.db')

def generate_and_save_graphs(conn):
    """
    Generate and save graphs using data from connected database, with optimizations for large datasets
    """
    output_dir = "graphs"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Get top artists by popularity (limit to top 50 for clearer visualizations)
    top_artists_query = """
    SELECT artist_id, artist_name, artist_popularity, artist_followers 
    FROM artists 
    ORDER BY artist_popularity DESC 
    LIMIT 50
    """
    top_artists = pd.read_sql_query(top_artists_query, conn)
    top_artist_ids = tuple(top_artists['artist_id'])

    # Get combined track data for only top artists
    tracks_query = f"""
    SELECT 
        t.track_id, t.track_name, t.popularity, t.duration_ms, t.explicit,
        t.time_signature, 
        a.album_name, a.release_date, a.album_image_url,
        ar.artist_name, ar.artist_popularity, ar.artist_followers,
        tf.danceability, tf.energy, tf.loudness, tf.acousticness,
        tf.instrumentalness, tf.liveness, tf.valence, tf.tempo
    FROM tracks t
    JOIN track_artists ta ON t.track_id = ta.track_id
    JOIN artists ar ON ta.artist_id = ar.artist_id
    JOIN albums a ON t.album_id = a.album_id
    JOIN track_features tf ON t.track_id = tf.track_id
    WHERE ar.artist_id IN {top_artist_ids}
    """
    tracks_df = pd.read_sql_query(tracks_query, conn)

    # 1. Artist Popularity Distribution
    popularity_query = """
    SELECT 
        ROUND(artist_popularity/10.0)*10 as popularity_bracket,
        COUNT(*) as count
    FROM artists
    GROUP BY ROUND(artist_popularity/10.0)*10
    ORDER BY popularity_bracket
    """
    popularity_dist = pd.read_sql_query(popularity_query, conn)
    
    fig = px.bar(
        popularity_dist,
        x="popularity_bracket",
        y="count",
        title="Artist Popularity Distribution",
        labels={
            "popularity_bracket": "Popularity Score (Brackets of 10)",
            "count": "Number of Artists"
        },
        template="plotly_dark"
    )
    fig.write_html(os.path.join(output_dir, "popularity_distribution.html"))

    # 2. Follower Distribution
    follower_query = """
    SELECT 
        ROUND(artist_popularity/10.0)*10 as popularity_bracket,
        AVG(artist_followers) as avg_followers,
        COUNT(*) as artist_count
    FROM artists
    GROUP BY ROUND(artist_popularity/10.0)*10
    ORDER BY popularity_bracket
    """
    follower_dist = pd.read_sql_query(follower_query, conn)
    
    fig = px.scatter(
        follower_dist,
        x="popularity_bracket",
        y="avg_followers",
        size="artist_count",
        title="Average Followers by Popularity Bracket",
        labels={
            "popularity_bracket": "Popularity Score (Brackets of 10)",
            "avg_followers": "Average Follower Count",
            "artist_count": "Number of Artists"
        },
        template="plotly_dark"
    )
    fig.update_layout(yaxis_type="log")
    fig.write_html(os.path.join(output_dir, "followers_by_popularity.html"))

    # 3. Audio Features Distribution
    feature_query = """
    WITH ArtistAverages AS (
        SELECT 
            ar.artist_id,
            AVG(tf.danceability) as avg_danceability,
            AVG(tf.energy) as avg_energy,
            AVG(tf.acousticness) as avg_acousticness,
            AVG(tf.valence) as avg_valence
        FROM artists ar
        JOIN track_artists ta ON ar.artist_id = ta.artist_id
        JOIN track_features tf ON ta.track_id = tf.track_id
        GROUP BY ar.artist_id
    )
    SELECT 
        'Danceability' as feature, avg_danceability as value FROM ArtistAverages
    UNION ALL
    SELECT 'Energy' as feature, avg_energy as value FROM ArtistAverages
    UNION ALL
    SELECT 'Acousticness' as feature, avg_acousticness as value FROM ArtistAverages
    UNION ALL
    SELECT 'Valence' as feature, avg_valence as value FROM ArtistAverages
    """
    feature_dist = pd.read_sql_query(feature_query, conn)
    
    fig = px.violin(
        feature_dist,
        x="feature",
        y="value",
        title="Distribution of Audio Features Across All Artists",
        box=True,
        template="plotly_dark"
    )
    fig.write_html(os.path.join(output_dir, "feature_distributions.html"))

    # 4. Top Artists Comparison
    top_artists_features = tracks_df.groupby('artist_name').agg({
        'danceability': 'mean',
        'energy': 'mean',
        'acousticness': 'mean',
        'valence': 'mean',
        'popularity': 'mean'
    }).round(3)

    fig = px.parallel_coordinates(
        top_artists_features.reset_index(),
        title="Audio Features of Top Artists",
        dimensions=['danceability', 'energy', 'acousticness', 'valence', 'popularity'],
        template="plotly_dark"
    )
    fig.write_html(os.path.join(output_dir, "top_artists_features.html"))

    # 5. Time Signatures
    time_sig_query = """
    SELECT 
        time_signature,
        COUNT(*) as count,
        AVG(popularity) as avg_popularity
    FROM tracks
    GROUP BY time_signature
    ORDER BY count DESC
    """
    time_sig_dist = pd.read_sql_query(time_sig_query, conn)
    
    fig = px.bar(
        time_sig_dist,
        x="time_signature",
        y="count",
        color="avg_popularity",
        title="Time Signature Distribution",
        labels={
            "time_signature": "Time Signature",
            "count": "Number of Tracks",
            "avg_popularity": "Average Popularity"
        },
        template="plotly_dark"
    )
    fig.write_html(os.path.join(output_dir, "time_signatures.html"))

    # 6. Release Year Trends
    year_query = """
    SELECT 
        SUBSTR(release_date, 1, 4) as year,
        COUNT(*) as track_count,
        AVG(t.popularity) as avg_popularity,
        AVG(tf.danceability) as avg_danceability,
        AVG(tf.energy) as avg_energy
    FROM albums a
    JOIN tracks t ON a.album_id = t.album_id
    JOIN track_features tf ON t.track_id = tf.track_id
    WHERE SUBSTR(release_date, 1, 4) >= '1900'
    GROUP BY SUBSTR(release_date, 1, 4)
    ORDER BY year
    """
    year_stats = pd.read_sql_query(year_query, conn)
    
    fig = px.line(
        year_stats,
        x="year",
        y=["avg_popularity", "avg_danceability", "avg_energy"],
        title="Trends Over Time",
        labels={
            "year": "Release Year",
            "value": "Average Value",
            "variable": "Metric"
        },
        template="plotly_dark"
    )
    fig.write_html(os.path.join(output_dir, "trends_over_time.html"))

In [41]:
generate_and_save_graphs(conn)